# ENV

In [1]:
%%capture
!pip install selenium
!pip install -y chromium-chromedriver

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from IPython.display import clear_output
import time

from tqdm.auto import tqdm
import pandas as pd
import gc

In [7]:
service = Service()

chrome_options = Options()
chrome_options.page_load_strategy = 'eager'
chrome_options.add_argument("--incognito")

chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("disable-infobars")

chrome_options.binary_location='/content/chromedriver.exe'
prefs = {'profile.default_content_setting_values': {'cookies': 2, 'images': 2, 'javascript': 2,
                            'plugins': 2, 'popups': 2, 'geolocation': 2,
                            'notifications': 2, 'auto_select_certificate': 2, 'fullscreen': 2,
                            'mouselock': 2, 'mixed_script': 2, 'media_stream': 2,
                            'media_stream_mic': 2, 'media_stream_camera': 2, 'protocol_handlers': 2,
                            'ppapi_broker': 2, 'automatic_downloads': 2, 'midi_sysex': 2,
                            'push_messaging': 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop': 2,
                            'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement': 2,
                            'durable_storage': 2}}
chrome_options.add_experimental_option('prefs', prefs)

# Crawl

In [8]:
# get thesis link

df = pd.DataFrame(
    {'Title': [], 'Link': [], 'Type': [], 'Abstract': [], 'Keyword': [], 'Citation': [], 'Author': [], 'Advisor': [],
     'Issue Date': [], 'Collection': [], 'PDF':[],'Check':[]})

def get_paper_list(link):
    cur_row=0
    driver = webdriver.Chrome(options=chrome_options)
    driver_1 = webdriver.Chrome(options=chrome_options)
    driver.get(link)
    collects = driver.find_element(By.XPATH, '/html/body/div[6]/div/div/div/div[2]/div').find_elements(By.TAG_NAME,'a')
    for i in collects:
        paper_list = i.get_attribute('href')
        driver_1.get(paper_list)
        pages=[page.get_attribute('name') for page in driver_1.find_elements(By.XPATH, '/html/body/div[5]/div[1]/div[2]/div/div/div/div[3]/ul/li/a')][1:-1]
        for page_num in pages:
            paper_list = i.get_attribute('href')+'?offset='+page_num
            driver_1.get(paper_list)
            items = driver_1.find_elements(By.XPATH, '/html/body/div[5]/div[1]/div[2]/div/div/div/table/tbody//strong/a')
            types = driver_1.find_elements(By.XPATH, '/html/body/div[5]/div[1]/div[2]/div/div/div/table/tbody//div[@class="col-md-12 browse-type"]/li')
            for j in range(len(items)):
                paper_link = items[j].get_attribute('href')
                clear_output(wait=True)
                print(paper_list)
                print(paper_link)
                df.loc[cur_row, 'Link'] = paper_link
                df.loc[cur_row, 'Collection'] = i.text
                df.loc[cur_row, 'Type'] = types[j].text
                df.loc[cur_row, 'Check'] = '0'
                cur_row = cur_row + 1

                display(df.tail())

            df.to_excel('lic.xlsx', index=True)

    driver.close()
    driver_1.close()
    return

get_paper_list('https://repository.vnu.edu.vn/handle/VNU_123/33109')
df.to_excel('lic.xlsx', index=True)

https://repository.vnu.edu.vn/handle/VNU_123/66901?offset=240
https://repository.vnu.edu.vn/handle/VNU_123/67053


,Title,Link,Type,Abstract,Keyword,Citation,Author,Advisor,Issue Date,Collection,PDF,Check
244,NaN,https://repository.vnu.edu.vn/handle/VNU_123/6...,Article,NaN,NaN,NaN,NaN,NaN,NaN,UET - Conference Papers,NaN,0
245,NaN,https://repository.vnu.edu.vn/handle/VNU_123/6...,Article,NaN,NaN,NaN,NaN,NaN,NaN,UET - Conference Papers,NaN,0
246,NaN,https://repository.vnu.edu.vn/handle/VNU_123/6...,Article,NaN,NaN,NaN,NaN,NaN,NaN,UET - Conference Papers,NaN,0
247,NaN,https://repository.vnu.edu.vn/handle/VNU_123/6...,Article,NaN,NaN,NaN,NaN,NaN,NaN,UET - Conference Papers,NaN,0
248,NaN,https://repository.vnu.edu.vn/handle/VNU_123/6...,Article,NaN,NaN,NaN,NaN,NaN,NaN,UET - Conference Papers,NaN,0


In [10]:
len(df)

249

In [15]:
# get thesis infor
driver = webdriver.Chrome(service=service,options=chrome_options)
for index in tqdm(range(len(df))):
    if df.loc[index,'Link']=='':
        continue
    if index%50==49:
        driver.close()
        driver = webdriver.Chrome(service=service,options=chrome_options)
    driver.get(df.loc[index,'Link'])
    df.loc[index,'Title'] = driver.find_element(By.XPATH, "/html/body/div[5]/div[2]/div/h2").text
    df.loc[index,'Year'] = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div[2]/div/div[1]/div[2]/div[2]/div[1]').text
    df.loc[index,'Author'] = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div[2]/div/div[1]/div[2]/div[2]/div[2]').text
    Advisor=driver.find_elements(By.XPATH, '/html/body/div[5]/div[2]/div/div[2]/div/div[1]/div[2]/div[2]/div[3]')
    if len(Advisor)!=0:
        df.loc[index,'Advisor'] = Advisor[0].text
    df.loc[index,'Abstract'] = driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div/div[2]/div/div[2]/div[1]/p').text
    Citations=driver.find_elements(By.XPATH, '/html/body/div[5]/div[2]/div/div[2]/div/div[2]/div[2]//span')
    if len(Citations)>=2:
        df.loc[index,'Keyword'] = Citations[0].text
        df.loc[index,'Citation'] = Citations[1].text
    else:
        df.loc[index,'Citation'] = Citations[0].text

    df[index,'Check']='1'
    if index%10==0:
        df.to_excel('lic.xlsx', index=False)

  0%|          | 0/249 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
df.to_excel('lic.xlsx', index=True)

In [ ]:
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://repository.vnu.edu.vn/handle/VNU_123/138324')

2


In [ ]:
tmp=driver.find_elements(By.XPATH, '//div[@class="okr-item-page-field-wrapper"]//div[@class="citation"]')
print(len(tmp))

for i in a:
  print(i.text)
  # break


Nguyen, T. N. T., et al. (2019). Air pollution monitoring network using low-cost sensors, a case study in Hanoi, Vietnam. In 4th International Forum on Sustainable Future in Asia 4th NIES International Forum, 23–24 January 2019, Pan Pacific Hanoi, Vietnam.
